# Crypto usage of qmrExchange

### Import required libraries

In [1]:
import sys
sys.path.append('..')
from main import *  

### Declare basic parameters for the simulation
qmrExchange allows for simulating multiple tickers at once, for statistical arbitrage and high-frequency-trading simulations. In the present case, we simulate 1 week worth of 1 minute data (24/7 trading).


In [2]:
time_interval = 'minute'
tickers = ['XYZ']
episodes = 1000

### Instantiate a Simulator

In [3]:
sim = Simulator(time_unit=time_interval, episodes=episodes)
sim.exchange.create_asset(tickers[0])
# create a mempool for the exchange
sim.exchange.crypto = True


### Add trading agents
- We add a naive market maker that creates both buy and sell orders in each period. It quotes buy and sell prices based on the last traded price and the specified spread percentage.
- We add a market taker that randomly buys and sells (based on the defined probabilities) on each period by means of market ordes (hence the word 'taker').

In [4]:

mm = CryptoMarketMaker(name='market_maker', tickers=tickers, aum=1_000, spread_pct=0.005, qty_per_order=4)
sim.add_agent(mm)

mt = CryptoMarketTaker(name='market_taker', tickers=tickers, aum=1_000, prob_buy=.2, prob_sell=.2, qty_per_order=1,seed=42)
sim.add_agent(mt)

print(sim.agents)

[<Agent: market_maker>, <Agent: market_taker>]


### Run the simulation

In [5]:
sim.run()

Retrieve all executed trades of our simulation

In [6]:
print(sim.exchange.mempool.transactions)
# print(sim.trades.head(20).to_markdown())

[<source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAAB90>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAAA90>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAABD0>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAAE90>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAAF10>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DA97D0>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DA9290>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAAC10>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DAB450>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DB4790>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DB5090>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DB5550>, <source.backend.MemPool.MempoolTransaction object at 0x000001EDF3DB5F50>, <source.backend.MemPool.MempoolTransa

Group asset price in fixed 15 Minute OHLCV Bars

In [7]:
df_15min = sim.get_price_bars(ticker=tickers[0],bar_size='15Min')
print(df_15min.head(20).to_markdown())

| dt                  |   open |   high |   low |   close |   volume |
|:--------------------|-------:|-------:|------:|--------:|---------:|
| 2022-01-01 00:00:00 | 100    | 100    | 99    |   99.5  |       14 |
| 2022-01-01 00:15:00 |  99.25 | 100    | 99.25 |   99.5  |       15 |
| 2022-01-01 00:30:00 |  99.25 |  99.25 | 98.75 |   98.75 |       27 |
| 2022-01-01 00:45:00 |  98.5  |  98.75 | 98    |   98.24 |       23 |
| 2022-01-01 01:00:00 |  97.99 |  97.99 | 97.51 |   97.51 |       12 |
| 2022-01-01 01:15:00 |  97.27 |  97.51 | 96.79 |   97.27 |       19 |
| 2022-01-01 01:30:00 |  97.51 |  97.51 | 97.03 |   97.27 |       15 |
| 2022-01-01 01:45:00 |  97.03 |  97.75 | 97.03 |   97.03 |       29 |
| 2022-01-01 02:00:00 |  96.79 |  97.99 | 96.79 |   97.99 |       14 |
| 2022-01-01 02:15:00 |  97.75 |  97.99 | 97.51 |   97.75 |       14 |
| 2022-01-01 02:30:00 |  97.51 |  97.75 | 97.03 |   97.51 |       24 |
| 2022-01-01 02:45:00 |  97.27 |  97.51 | 96.55 |   97.03 |       23 |
| 2022

Retrieve a dataframe of an agents holding at each period of time

In [8]:
mt_holdings = sim.get_portfolio_history('market_taker')
mm_holdings = sim.get_portfolio_history('market_maker')

print(mt_holdings.head(20).to_markdown())

|                     |              XYZ |     cash |              aum |
|:--------------------|-----------------:|---------:|-----------------:|
| 2022-01-01 00:00:00 |      0           |    0     |      0           |
| 2022-01-01 00:01:00 |      0           |    0     |      0           |
| 2022-01-01 00:02:00 |      0           |    0     |      0           |
| 2022-01-01 00:03:00 |      0           |    0     |      0           |
| 2022-01-01 00:04:00 |      0           |    0     |      0           |
| 2022-01-01 00:05:00 |   9975           | -100.001 |   9875           |
| 2022-01-01 00:06:00 |  19800           | -200.002 |  19600           |
| 2022-01-01 00:07:00 |  69102           |   97.997 |  69200           |
| 2022-01-01 00:08:00 | 118703           |  395.996 | 119099           |
| 2022-01-01 00:09:00 | 178428           |  594.744 | 179023           |
| 2022-01-01 00:10:00 | 247755           |  693.991 | 248449           |
| 2022-01-01 00:11:00 | 317082           |  793.238

### Plot the results

In [10]:
from source.backend.helpers import plot_bars
df_15min = sim.get_price_bars(ticker=tickers[0], bar_size='15Min')
plot_bars(df_15min)

In [11]:
import plotly.express as px
import pandas as pd

df_plot = pd.DataFrame()
df_plot['Market Maker'] = mm_holdings['aum']
df_plot['Market Taker'] = mt_holdings['aum']
fig = px.line(df_plot,labels={'variable':'Agents','value':'Assets Under Management','dt':'Date'})
fig.show()